In [ ]:
!git clone https://github.com/zhandauov/alice_catch_me.git

Cloning into 'alice_catch_me'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 0), reused 10 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [ ]:
import numpy as np
import pandas as pd
import pickle

In [ ]:
path_train = 'alice_catch_me/train_sessions.csv'
path_test = 'alice_catch_me/test_sessions.csv'

In [ ]:
train_data = pd.read_csv(path_train)
train_data.shape

(253561, 22)

In [ ]:
def transform_data(data):
    for i in range(1,11):
        temp_name = 'time' + str(i)
        data[temp_name] = pd.to_datetime(data[temp_name], format = '%Y-%m-%d %H:%M:%S')

    data['start_hour'] = data['time1'].dt.hour
    data['s_duration'] = np.zeros((data.shape[0],))

    data['s_duration'] = (data['time10'] - data['time1']) / np.timedelta64(1, 'm')
    for i in range(9,0,-1):
        temp_name = 'time' + str(i)
        data.loc[data['s_duration'].isnull() == True, 's_duration'] = (data[temp_name] - data['time1']) / np.timedelta64(1, 'm')
    data['s_duration'] = data['s_duration'].round(2)

    for i in range(10,0,-1):
        temp_name = 'time' + str(i)
        data.drop([temp_name], axis=1, inplace = True)

    data.drop(['session_id'], axis=1, inplace = True)

    data.fillna(0, inplace=True)

    for i in range(1,11):
        sitename = 'site' + str(i)
        data[sitename] = data[sitename].astype('int64')
    return data

In [ ]:
train_data.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,target,start_hour,s_duration
0,718,0,0,0,0,0,0,0,0,0,0,10,0.00
1,890,941,3847,941,942,3846,3847,3846,1516,1518,0,11,0.43
2,14769,39,14768,14769,37,39,14768,14768,14768,14768,0,16,0.12
3,782,782,782,782,782,782,782,782,782,782,0,10,4.50
4,22,177,175,178,177,178,175,177,177,178,0,10,4.10


In [ ]:
with open('alice_catch_me/site_dic.pkl', 'rb') as f:
    site_dictionary = pickle.load(f)

In [ ]:
len(site_dictionary)

48371

In [ ]:
site_dictionary

In [ ]:
def find_site_byid(id):
    for key, value in site_dictionary.items():
        if value == id:
            print(key)

## Machine Learning time! <3

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.asarray(train_data.drop(['target'], axis = 1)), np.asarray(train_data['target']), test_size=0.20, random_state=42)

In [ ]:
classifier = XGBClassifier()
classifier.fit(np.asarray(train_data.drop(['target'], axis = 1)), np.asarray(train_data['target']))

D:\Programs\Anaconda\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:05:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
y_pred

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [ ]:
accuracy_score(y_pred, y_test)

0.9929406661013941

## Submission

In [ ]:
test_set = pd.read_csv('test_sessions.csv')
test_set.shape

In [ ]:
test_set

In [ ]:
sub_pred = classifier.predict(np.asarray(test_set))

In [ ]:
test_sites = test.iloc[:,:10]
test_times = test.iloc[:,10:]
preds = model.predict(x=[test_sites, test_times])

In [ ]:
test_set = pd.read_csv('alice_catch_me/test_sessions.csv')

In [ ]:
test_set = test_set['session_id']

In [ ]:
test_set

0            1
1            2
2            3
3            4
4            5
         ...  
82792    82793
82793    82794
82794    82795
82795    82796
82796    82797
Name: session_id, Length: 82797, dtype: int64

In [ ]:
preds = pd.DataFrame(preds)

In [ ]:
preds = pd.concat([test_set, preds], axis = 1)

In [ ]:
preds

,session_id,0
0,1,2.152496e-06
1,2,4.303819e-16
2,3,7.105600e-17
3,4,1.215825e-15
4,5,4.034104e-09
...,...,...
82792,82793,1.591522e-12
82793,82794,2.711733e-14
82794,82795,3.669322e-03
82795,82796,1.159063e-11


In [ ]:
preds.to_csv('submission_file.csv', float_format='%.15f', index=False)

In [ ]:
preds.columns = ['session_id', 'target']

In [ ]:
preds.to_csv('new_pred.csv', index = False)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.bar(range(len(classifier.feature_importances_)), classifier.feature_importances_)
plt.show()

### Word embeddings & dense layers

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
train = pd.read_csv(path_train)
train = transform_data(train)
test = pd.read_csv(path_test)
test = transform_data(test)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train.drop("target", axis=1), train['target'], test_size=0.20, random_state=42)

In [ ]:
X_train_sites = X_train.iloc[:,:10]
X_train_times = X_train.iloc[:,10:]
X_test_sites = X_test.iloc[:,:10]
X_test_times = X_test.iloc[:,10:]

In [ ]:
X_train_times.iloc[0].shape

(2,)

In [ ]:
len(site_dictionary)

In [ ]:
# define two sets of inputs
inputA = keras.layers.Input(shape=(10,))
inputB = keras.layers.Input(shape=(2,))
# the first branch operates on the first input
x = keras.layers.Embedding(48371+1, 2)(inputA)
x = keras.layers.Flatten()(x)
x = keras.Model(inputs=inputA, outputs=x)
# the second branch opreates on the second input
y = keras.layers.Dense(16, activation="relu")(inputB)
y = keras.layers.Dense(4, activation="relu")(y)
y = keras.Model(inputs=inputB, outputs=y)
# combine the output of the two branches
combined = keras.layers.Concatenate()([x.output, y.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
z = keras.layers.Dense(64, activation="relu")(combined)
z = keras.layers.Dense(32, activation="relu")(z)
z = keras.layers.Dense(1, activation="sigmoid")(z)
# our model will accept the inputs of the two branches and
# then output a single value
model = keras.Model(inputs=[x.input, y.input], outputs=z)

In [ ]:
keras.utils.plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 2)]          0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 10, 2)        96744       input_7[0][0]                    
__________________________________________________________________________________________________
dense_15 (Dense)                (None, 16)           48          input_8[0][0]                    
___________________________________________________________________________________________

In [ ]:
model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.01),
              loss='binary_crossentropy', metrics = ['accuracy', 'AUC'])

In [ ]:
model.fit(
    x=[X_train_sites, X_train_times], y=y_train,
    validation_data=([X_test_sites, X_test_times], y_test),
    epochs=10, verbose = 1)

Epoch 1/10
6339/6339 [==============================] - 21s 3ms/step - loss: 0.0473 - accuracy: 0.9899 - auc: 0.8360 - val_loss: 0.0334 - val_accuracy: 0.9910 - val_auc: 0.9108
Epoch 2/10
6339/6339 [==============================] - 20s 3ms/step - loss: 0.0290 - accuracy: 0.9917 - auc: 0.9523 - val_loss: 0.0313 - val_accuracy: 0.9919 - val_auc: 0.9151
Epoch 3/10
6339/6339 [==============================] - 21s 3ms/step - loss: 0.0262 - accuracy: 0.9924 - auc: 0.9593 - val_loss: 0.0295 - val_accuracy: 0.9924 - val_auc: 0.9370
Epoch 4/10
6339/6339 [==============================] - 20s 3ms/step - loss: 0.0248 - accuracy: 0.9928 - auc: 0.9601 - val_loss: 0.0341 - val_accuracy: 0.9921 - val_auc: 0.9158
Epoch 5/10
6339/6339 [==============================] - 21s 3ms/step - loss: 0.0234 - accuracy: 0.9931 - auc: 0.9692 - val_loss: 0.0325 - val_accuracy: 0.9924 - val_auc: 0.9131
Epoch 6/10
6339/6339 [==============================] - 21s 3ms/step - loss: 0.0227 - accuracy: 0.9932 - auc: 0.968

In [ ]:
X_train_times.shape

(202848, 2)

In [ ]:
model.save_weights("model.h5")